In [ ]:
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np

from ultralytics import YOLO

# Video to test trackreid on

### Let's try to detect jungling balls

In [ ]:
VIDEO_PATH = "../../data/video/jungling_guy.mp4"

In [ ]:

from IPython.display import Video; Video(VIDEO_PATH, width=1024, height=600)


# Video to frames

In [ ]:
# Transform this VIDEO_PATH into a list of frames in this folder under frames/
!mkdir -p frames
!ffmpeg -i $VIDEO_PATH -vf fps=12 frames/%d.png -hide_banner -loglevel panic

In [ ]:
available_frames = glob.glob("frames/*.png")
available_frames = sorted(available_frames, key=lambda x: int(x.split("/")[-1].split(".")[0]))

# Detection

In [ ]:
model = YOLO("yolov8n.pt")
results = model(available_frames[0])[0]

In [ ]:
%matplotlib inline
plt.imshow(cv2.cvtColor(results.plot(), cv2.COLOR_BGR2RGB))
plt.show()

### We will track only sports ball so we need to filter them out

In [ ]:
SPORTS_BALL_CLASS_ID = 32

## Tracking without trackreid

In [ ]:
from bytetracker import BYTETracker
from bytetracker.basetrack import BaseTrack
from utils import draw_all_bbox_on_image, yolo_results_to_bytetrack_format, yolo_results_with_track_id

In [ ]:
tracker = BYTETracker(track_thresh= 0.3, track_buffer = 5, match_thresh = 0.85, frame_rate= 12)
BaseTrack._count = 0
model = YOLO("yolov8n.pt", task="detect")

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
OUTPUT_NO_REID = "output_no_reid.mp4"
out = cv2.VideoWriter(OUTPUT_NO_REID, fourcc, 12, (640, 480))

for frame_id, image_filename in enumerate(available_frames):
    detections = model(image_filename, classes=[SPORTS_BALL_CLASS_ID])[0]
    detections_bytetrack_format = yolo_results_to_bytetrack_format(detections)
    tracked_objects = tracker.update(detections_bytetrack_format, frame_id)